<a href="https://colab.research.google.com/github/sequeirayeslin/digit_recog_ann_from_scratch/blob/main/Building_digit_recog_ann_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from math import e

In [ ]:
unipolar_sigmoid = lambda x: 1/(1+e**(-x))
unipolar_sigmoid_diff = lambda x: x*(1-x)


act_dict={
    "unipolar_sigmoid": [unipolar_sigmoid, unipolar_sigmoid_diff]
    #, "relu": [relu, relu_diff]
}

In [ ]:
default_w_limit = 1
default_act = "relu"
biasInput = 1

class Neuron:
  def __init__(self,inputs, w_limit=default_w_limit, act=default_act):
    
    self.W = np.random.rand(inputs+1)*w_limit *2 - w_limit
    #self.W = np.random.rand(inputs)*w_limit *2 - w_limit

    self.act, self.act_diff = act_dict[act]
  
  def net(self, X):

    X = np.append(X, biasInput)

    return np.dot(self.W, X)
  
  def out(self, X):

    net = self.net(X)
    return self.act(net)

class Layer:

  def __init__(self, neurons, inputs, w_limit=default_w_limit, act=default_act):

    self.neurons = [Neuron(inputs,w_limit,act) for _ in range(neurons)]
    self.act = act_dict[act][0]
    self.act_diff = act_dict[act][1]

    self.out_mem = None
    self.diffs = None
    self.error= None
    self.in_mem = None

  def out(self, X):

    self.in_mem = np.append(X, biasInput)
    #self.in_mem = np.array(X)

    out = [neuron.out(X) for neuron in self.neurons]
    out = np.array(out)

    self.out_mem = out
    self.diffs = self.act_diff(out)

    return out

class Network:

  def __init__(self):
    self.layers=[]

  def out(self, X):

    in_data = X

    for layer in self.layers:
      out = layer.out(in_data)
      in_data = out
    
    return out

  def addInLayer(self, neurons, inputs, w_limit=default_w_limit, act=default_act):
    self.layers.append(Layer(neurons, inputs, w_limit, act))

  def addLayer(self, neurons, w_limit=default_w_limit, act=default_act):
    inputs = len(self.layers[-1].neurons)
    self.layers.append(Layer(neurons, inputs, w_limit, act))

  def calc_error(self,Y):
    
    for layer in reversed(self.layers):

      if layer == self.layers[-1]:
        layer.error = (Y-layer.out_mem)

      else:
        layer.error = []
        for i in range(len(layer.neurons)):
          temp = [neuron.W[i] for neuron in last_layer.neurons]
          temp = temp * last_layer.error * last_layer.diffs
          temp = np.sum(temp)
          layer.error.append(temp)
        layer.error = np.array(layer.error)

      last_layer = layer


  def train(self,X,Y,C):

    X = np.array(X)
    Y = np.array(Y)

    self.out(X)
    self.calc_error(Y)

    for layer in self.layers:
      for i,neuron in enumerate(layer.neurons):
        neuron.W = neuron.W + C * layer.diffs[i] * layer.error[i] * layer.in_mem
  
  def test_classifier(self, X, Y):

    X = np.array(X)
    Y = np.array(Y)
    
    total = len(X)
    correct = 0

    for x,y in zip(X,Y):

      out = self.out(x)
      isItCorrect = np.where(y == np.amax(y)) [0] == np.where(out == np.amax(out)) [0]
      isItCorrect = int(isItCorrect)
      correct += isItCorrect
    
    print('{}% accuracy'.format(correct/total * 100))

In [ ]:
#importing ONLY handwritten number detection training set from tenserflow

from tensorflow.keras.datasets import mnist

(x_train,y_train),(x_test,y_test)=mnist.load_data()

y_train = [[int(y==num) for num in range(10)]for y in y_train]  
x_train = [x.flatten()/255 for x in x_train]

y_test = [[int(y==num) for num in range(10)]for y in y_test]
x_test = [x.flatten()/255 for x in x_test]

In [ ]:
brain = Network()
brain.addInLayer(neurons = 30, inputs = 784)
brain.addLayer(neurons=10)

In [ ]:
from IPython.display import clear_output

In [ ]:
epochs = 1

for i in range(epochs):

  clear_output()
  i_10 = round(i/epochs * 10)

  print('Training (epoch {})... \n['.format(i+1) + '--'*(i_10) + '->' + '  '*(10-(i_10+1)) + ']', end = '')

  for x,y in zip(x_train, y_train):
    brain.train(x,y,C=0.01)

clear_output()
print('Traning complete! \n[--------------------]')

Traning complete! 
[--------------------]


In [ ]:
brain.test(x_test, y_test)

'95.07% accuracy'

In [ ]:
brain.test(x_train, y_train)

'96.42166666666667% accuracy'